In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from bayes_opt import BayesianOptimization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt

data = pd.read_excel('3.xlsx')
x = data.iloc[:, :2].values  # 修改特征列数量
y = data.iloc[:, -1].values  # 目标列

scaler = StandardScaler()
x = scaler.fit_transform(x)

activation_map = {0: 'relu', 1: 'sigmoid', 2: 'tanh'}

def create_model(hidden_layers, neurons, learning_rate, activation_code, dropout_rate=0.5, l2_reg=0.01):
    activation = activation_map[int(activation_code)]
    model = Sequential()
    model.add(Dense(int(neurons), input_dim=x.shape[1], activation=activation, kernel_regularizer=l2(l2_reg)))
    for _ in range(int(hidden_layers)):
        model.add(Dense(int(neurons), activation=activation, kernel_regularizer=l2(l2_reg)))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='linear'))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mse'])
    return model

def optimize(hidden_layers, neurons, learning_rate, activation_code):
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    r2_scores = []
    for train_idx, val_idx in kfold.split(x):
        train_x, val_x = x[train_idx], x[val_idx]
        train_y, val_y = y[train_idx], y[val_idx]
        
        model = create_model(hidden_layers, neurons, learning_rate, activation_code)
        model.fit(train_x, train_y, epochs=200, batch_size=32, verbose=0)
        predictions = model.predict(val_x).flatten()
        r2 = r2_score(val_y, predictions)
        r2_scores.append(r2)
    return np.mean(r2_scores)

best_r2 = -np.inf
best_params = None

def track_optimize(hidden_layers, neurons, learning_rate, activation_code):
    global best_r2, best_params
    r2 = optimize(hidden_layers, neurons, learning_rate, activation_code)
    if r2 > best_r2:
        best_r2 = r2
        best_params = {
            'hidden_layers': hidden_layers,
            'neurons': neurons,
            'learning_rate': learning_rate,
            'activation': activation_map[int(activation_code)]
        }
    return r2

optimizer = BayesianOptimization(
    f=track_optimize, 
    cv=10,
    pbounds={
        'hidden_layers': (1, 5),
        'neurons': (10, 200),
        'learning_rate': (0.0001, 0.1),
        'activation_code': (0, 2)
    },
    random_state=42,
)

optimizer.set_gp_params(alpha=1e-4)
optimizer.maximize(n_iter=100000, init_points=10)

if best_params is None:
    raise ValueError("最佳参数未正确更新。请检查优化目标函数。")

best_model = create_model(
    hidden_layers=best_params['hidden_layers'],
    neurons=best_params['neurons'],
    learning_rate=best_params['learning_rate'],
    activation_code=int(list(activation_map.keys())[list(activation_map.values()).index(best_params['activation'])])
)
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.1, random_state=42)
best_model.fit(train_x, train_y, epochs=200, batch_size=32, verbose=0) 
train_pred = best_model.predict(train_x).flatten()
test_pred = best_model.predict(test_x).flatten()

train_mse = mean_squared_error(train_y, train_pred)
train_rmse = np.sqrt(train_mse)
train_mae = mean_absolute_error(train_y, train_pred)
train_r2 = r2_score(train_y, train_pred)

test_mse = mean_squared_error(test_y, test_pred)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(test_y, test_pred)
test_r2 = r2_score(test_y, test_pred)

print(f'Train - MSE: {train_mse}, RMSE: {train_rmse}, MAE: {train_mae}, R^2 score: {train_r2}')
print(f'Test - MSE: {test_mse}, RMSE: {test_rmse}, MAE: {test_mae}, R^2 score: {test_r2}')
print(f'Best parameters found: {best_params}')
print(f'Best R^2 score during optimization: {best_r2}')
